In [95]:
import pickle
import contextlib
import pandas as pd

def dump_pickle(file, outdir):
    """pickle a file to output directory
    """
    f = open(outdir,"wb")
    pickle.dump(file, f)
    
def load_pickle(indir):
    """ load a pickle from indir
    """
    f = open(indir,"rb")
    return pickle.load(f)
    
def multiple_strreplace(string, replace_dic):
    for k,v in replace_dic.items():
        string = string.replace(k,v)
    return string

def parse_queries(qa_fp, replace_dic):
    qa = pd.read_csv(qa_fp)
    queries = list(map(lambda query: multiple_strreplace(query, replace_dic), qa['question'].values))
    return queries
    
def log(content, logpath):
      try:
        content = content.replace('<s>[INST] <<SYS>>', 'System role:'),
        content = content.replace('<</SYS>>', ''),
        content = content.replace('[/INST]', '\\n'),
      except:
          pass
      if os.path.exists(logpath):
          with open(logpath, 'a') as f:
              with contextlib.redirect_stdout(f):
                  print(content)
      else:
          with open(logpath, 'w') as f:
              with contextlib.redirect_stdout(f):
                 print(content)

In [96]:
import os
from dotenv import load_dotenv
#from huggingface_hub import login
from pathlib import Path
dotenv_path = Path('../keys/.env')
load_dotenv(dotenv_path=dotenv_path)
#hf_token = os.getenv("HF_TOKEN")
#os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token
#os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
#os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO']='0.0'

True

## Retriever

In [28]:
import os
import torch

from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings, HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever, ContextualCompressionRetriever
from langchain_community.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_chroma import Chroma
from langchain.storage import InMemoryStore
from ragatouille import RAGPretrainedModel

from langchain.llms import HuggingFaceEndpoint, HuggingFaceHub
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [97]:
querys = load_pickle('./querys.pck')

In [98]:
querys

['What kind of products or services is NVDIA providing?',
 'Who are the customers of NVDIA or what types of markets are NVDIA operating in?',
 'Who are the competitors of NVDIA?',
 "What are the risk factors and uncertainties that could affect the NVDIA's future performance?",
 'What is the 2025 revenue of NVDIA?',
 'What is the 2024 revenue of NVDIA?',
 'What is the 2025 total liabilities?',
 "What is the 2025 total shareholders' equity?",
 'What is the 2025 total current assets?',
 'What is the 2025 total current liabilities?',
 'What is the 2025 gross margin?']

In [32]:
file_path = "../data/nvda-20250126.pdf"
data = PDFMinerLoader(file_path, mode='page')
content = data.load()
print(len(content), len(content[0].page_content))

118 5167


In [88]:
# colbert
#max_document_length = 100 didn't work
max_document_length = 512
colbert_k = 5
documents = list(map(lambda x: x.page_content, content))
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
RAG.index(
    collection=documents,
    index_name="NVDIA",
    max_document_length=max_document_length,
    split_documents=True,
)
clbt_retriever = RAG.as_langchain_retriever(k=colbert_k)

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Mar 09, 03:25:42] #> Note: Output directory .ragatouille/colbert/indexes/NVDIA already exists


[Mar 09, 03:25:42] #> Will delete 10 files already at .ragatouille/colbert/indexes/NVDIA in 20 seconds...
[Mar 09, 03:26:03] [0] 		 #> Encoding 1342 passages..


  0%|                                                    | 0/42 [00:00<?, ?it/s]/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|███████████████████████████████████████████| 42/42 [02:13<00:00,  3.18s/it]

[Mar 09, 03:28:17] [0] 		 avg_doclen_est = 57.59687042236328 	 len(local_sample) = 1,342


[Mar 09, 03:28:17] [0] 		 Creating 4,096 partitions.
[Mar 09, 03:28:17] [0] 		 *Estimated* 77,295 embeddings.
[Mar 09, 03:28:17] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/NVDIA/plan.json ..
used 20 iterations (47.4364s) to cluster 73431 items into 4096 clusters
[0.032, 0.035, 0.033, 0.031, 0.033, 0.034, 0.032, 0.033, 0.031, 0.034, 0.03, 0.032, 0.035, 0.036, 0.033, 0.033, 0.029, 0.03, 0.032, 0.033, 0.033, 0.035, 0.031, 0.033, 0.03, 0.031, 0.034, 0.031, 0.034, 0.035, 0.033, 0.033, 0.034, 0.032, 0.033, 0.029, 0.034, 0.035, 0.033, 0.035, 0.032, 0.031, 0.032, 0.035, 0.032, 0.032, 0.03, 0.037, 0.036, 0.032, 0.03, 0.033, 0.034, 0.031, 0.032, 0.032, 0.038, 0.035, 0.038, 0.032, 0.03, 0.033, 0.032, 0.033, 0.035, 0.035, 0.035, 0.033, 0.031, 0.033, 0.034, 0.029, 0.032, 0.035, 0.03, 0.034, 0.034, 0.033, 0.031, 0.035, 0.034, 0.034, 0.031, 0.033, 0.034, 0.032, 0.032, 0.032, 0.031, 0.035, 0.033, 0.036, 0.032, 0.033, 0.033, 0.034, 0.037, 0.033, 0.032, 0.033, 0.034, 0.036, 0.031

...

[Mar 09, 03:29:05] [0] 		 #> Encoding 1342 passages..



100%|███████████████████████████████████████████| 42/42 [02:09<00:00,  3.08s/it]
1it [02:11, 131.74s/it]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 567.56it/s]

[Mar 09, 03:31:17] #> Optimizing IVF to store map from centroids to list of pids..
[Mar 09, 03:31:17] #> Building the emb2pid mapping..
[Mar 09, 03:31:17] len(emb2pid) = 77295



100%|████████████████████████████████████| 4096/4096 [00:00<00:00, 48079.11it/s]

[Mar 09, 03:31:17] #> Saved optimized IVF to .ragatouille/colbert/indexes/NVDIA/ivf.pid.pt
Done indexing!


In [89]:
# bm25
chunk_size = 800
chunk_overlap = 30
# chunk_size, chunk_overlap = 100, 20 did not work
keyword_retriever_k = 5
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                          chunk_overlap=chunk_overlap)
chunks = splitter.split_documents(content)
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k = keyword_retriever_k

In [90]:
# hybrid
retrievers = [clbt_retriever, keyword_retriever]
retriever_weights = [0.3, 0.7]
ensemble_retriever = EnsembleRetriever(retrievers= retrievers,
                                       weights=retriever_weights)

In [91]:
# reranker
rerank_model_name = "BAAI/bge-reranker-base"
rerank_top_n = 3
rerank_model = HuggingFaceCrossEncoder(model_name=rerank_model_name)
compressor = CrossEncoderReranker(model=rerank_model, top_n=rerank_top_n)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

## Generation

In [ ]:
#relevant_context = compression_retriever.get_relevant_documents(querys[0])

didn't work. too slow.
from langchain_huggingface import HuggingFacePipeline
import torch
#model_id = "HuggingFaceH4/zephyr-7b-alpha"
#model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
#model_id = 'deepseek-ai/DeepSeek-R1'
#model_id = 'meta-llama/Llama-3.1-8B'
model_id = 'meta-llama/Llama-3.2-1B'
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.9,
        "do_sample":True,
        'top_p':0.95,
        #'torch_dtype':torch.bfloat16, 
        'device_map':"auto"
    },
)

from openai import OpenAI
client = OpenAI()
models = client.models.list()
for model in models:
    print(model.id)

In [44]:
# https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18")

In [92]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """
<|system|>>
You are a Financial Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)

In [93]:
output_parser = StrOutputParser()
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

### Test question: What is the total revenue of NVIDIA for fiscal year 2025?

In [48]:
# no rag
response = llm.invoke(query)
print(response.content)

As of my last update, I do not have access to real-time financial data or projections beyond October 2023. To obtain the most accurate and up-to-date revenue figures for NVIDIA for fiscal year 2025, I recommend checking their official financial reports, investor relations website, or financial news sources.


In [53]:
# with rag
rag_response = chain.invoke(query)
print(rag_response)
# This is correct.

The revenue of NVIDIA in fiscal year 2025 was $130.5 billion.


### Test question: What is the debt-to-equity ratio of NVIDIA for fiscal year 2025?

In [59]:
# test if llm knows how to calculate debt-to-equity ratio
d2e_response = llm.invoke('How to calculate debt-to-equity ratio?')
print(d2e_response.content)

The debt-to-equity (D/E) ratio is a financial metric used to assess a company's financial leverage by comparing its total liabilities to its shareholders' equity. It indicates how much debt a company is using to finance its assets relative to the equity provided by shareholders.

### Formula:

The debt-to-equity ratio is calculated using the following formula:

\[
\text{Debt-to-Equity Ratio} = \frac{\text{Total Liabilities}}{\text{Total Equity}}
\]

### Steps to Calculate:

1. **Find Total Liabilities**: This includes all of a company's debts, such as loans, mortgages, and any other financial obligations. You can usually find this information on the company's balance sheet.

2. **Find Total Equity**: This includes all the ownership equity in the company, which can consist of common stock, preferred stock, and retained earnings. This information is also found on the balance sheet.

3. **Plug the values into the formula**: Substitute the total liabilities and total equity into the formul

In [55]:
# no rag response
response = llm.invoke('What is the debt-to-equity ratio of NVIDIA as of January 2025?')
print(response.content)

I’m unable to provide real-time financial data or projections for specific companies such as NVIDIA as of January 2025. To find the most accurate and up-to-date information on NVIDIA's debt-to-equity ratio, I recommend checking financial news websites, the company's investor relations page, or financial databases.


In [57]:
# with rag and zero-shot propmt 
rag_response = chain.invoke('What is the debt-to-equity ratio of NVIDIA as of January 2025?')
print(rag_response)

I don't know.


In [71]:
# with rag and chain of thoughts
# did not work
query = """Q: What is the debt-to-equity ratio of company A?
           A: The debt-to-equity (D/E) ratio is a financial metric used to assess a company's financial leverage by comparing its total liabilities to its shareholders' equity.
           ### Formula:
            \[
            \text{Debt-to-Equity Ratio} = \frac{\text{Total Liabilities}}{\text{Total Equity}}
            \]

           ### Steps to Calculate:
               1. **Find Total Liabilities of company A**: $500,000
               2. **Find Total Equity of company A**: $250,000
               3. **Plug the values into the formula**: Substitute the total liabilities and total equity into the formula to calculate the D/E ratio.
            \[
            \text{D/E Ratio} = \frac{500,000}{250,000} = 2.0
            \]
            In this example, the debt-to-equity ratio of company A is 2.0
            Q: What is the debt-to-equity ratio of NVIDIA as of January 2025?"""
rag_response = chain.invoke(query)
print(rag_response)

I don't know.


In [76]:
# with rag
rag_response = chain.invoke('What is the 2025 total liabilities?')
print(rag_response)
# This is correct.

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


The total liabilities for NVIDIA Corporation as of January 26, 2025, is $32,274 million.


In [80]:
# with rag
rag_response = chain.invoke("What is the 2025 total shareholders' equity?")
print(rag_response)
# This is wrong. Get the number one line above.

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


The total shareholders' equity as of January 26, 2025, is $68,038 million.


In [81]:
# with rag and instruction
rag_response = chain.invoke("First, look for the 2025 total shareholders' equity. Second, look for the 2025 total liabilities. Given these two numbers, calculate the debt-to-equity ratio of NVIDIA as of January 2025?")
print(rag_response)
# This is wrong. $79,327 million is Total Shareholders' Equity and one line above 'Total liabilites and shareholder's equity'.

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


The total shareholders' equity for NVIDIA as of January 26, 2025, is $68,038 million, and the total liabilities are $79,327 million. 

To calculate the debt-to-equity ratio:

Debt-to-Equity Ratio = Total Liabilities / Total Shareholders' Equity  
Debt-to-Equity Ratio = $79,327 million / $68,038 million  
Debt-to-Equity Ratio ≈ 1.165  

Thus, the debt-to-equity ratio of NVIDIA as of January 2025 is approximately 1.17.


In [86]:
# with rag and instruction
rag_response = chain.invoke("Look for necessay information from given documents and calculate the debt-to-equity ratio of NVIDIA as of January 2025.")
print(rag_response)
# This is wrong. $79,327 million is Total Shareholders' Equity and one line above 'Total liabilites and shareholder's equity'.

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


To calculate the debt-to-equity ratio, we can use the formula:

\[
\text{Debt-to-Equity Ratio} = \frac{\text{Total Liabilities}}{\text{Total Shareholders' Equity}}
\]

From the provided information as of January 26, 2025:

- Total Liabilities = $32,274 million (from current liabilities) + $8,463 million (long-term debt) + additional long-term liabilities not specified = $80,126 million
- Total Shareholders' Equity = $68,038 million

Now we can calculate:

\[
\text{Debt-to-Equity Ratio} = \frac{80,126}{68,038} \approx 1.18
\]

Therefore, the debt-to-equity ratio of NVIDIA as of January 2025 is approximately 1.18.


In [94]:
# refine retriever parameters and instruction
rag_response = chain.invoke("First, look for the 2025 total shareholders' equity. Second, look for the 2025 total liabilities. Calculate the debt-to-equity ratio of NVIDIA as of January 2025.")
print(rag_response)
# This is wrong. $79,327 million is Total Shareholders' Equity and one line above 'Total liabilites and shareholder's equity'.

Loading searcher for index NVDIA for the first time... This may take a few seconds
[Mar 09, 03:43:21] #> Loading codec...
[Mar 09, 03:43:21] #> Loading IVF...
[Mar 09, 03:43:21] #> Loading doclens...


/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1613.19it/s]

[Mar 09, 03:43:21] #> Loading codes and residuals...



100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 115.21it/s]
/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: First, look for the 2025 total shareholders' equity. Second, look for the 2025 total liabilities. Calculate the debt-to-equity ratio of NVIDIA as of January 2025., 		 True, 		 None
#> Output IDs: torch.Size([33]), tensor([  101,     1,  2034,  1010,  2298,  2005,  1996, 16798,  2629,  2561,
        15337,  1005, 10067,  1012,  2117,  1010,  2298,  2005,  1996, 16798,
         2629,  2561, 22393, 14680,  1012, 18422,  1996,  7016,  1011,  2000,
         1011, 10067,   102])
#> Output Mask: torch.Size([33]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])

The total shareholders' equity for NVIDIA as of January 26, 2025, is $80,126 million (as stated in the first document). 

However, the total liabilities for January 26, 2025, is not directly provided in the given context. Thus, I don't know the total liabilities for NV

### Test question: What is the current ratio of NVIDIA for fiscal year 2025?

In [84]:
rag_response = chain.invoke("First, look for the 2025 total current liabilities (instead of total liabilites). Second, look for the 2025 total current assets. Given these two numbers, calculate the current ratio of NVIDIA as of January 2025?")
print(rag_response)
# This is correct.

/Users/Nan/miniconda3/envs/rag/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


The total current liabilities for NVIDIA as of January 26, 2025, are $18,047 million. The total current assets for the same date are $80,126 million.

To calculate the current ratio, you use the formula:

\[
\text{Current Ratio} = \frac{\text{Total Current Assets}}{\text{Total Current Liabilities}}
\]

Substituting in the values:

\[
\text{Current Ratio} = \frac{80,126}{18,047} \approx 4.44
\]

Thus, the current ratio of NVIDIA as of January 2025 is approximately 4.44.
